In [1]:
!pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


## Agents

In [2]:
from google.colab import userdata
from autogen import ConversableAgent

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
llm_config={
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get('OPENAI_API_KEY'),
    "temperature": 0.7
}

In [4]:
audience = ConversableAgent(
    name="Audience",
    llm_config=llm_config,
    is_termination_msg=lambda msg: "HAHA" in msg['content'].upper(),
    system_message='You are a member of the audience of a comedy show that is hard to impress'
)

[autogen.oai.client: 10-01 04:18:52] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [5]:
comedian = ConversableAgent(
    name="Comedian",
    llm_config=llm_config,
    max_consecutive_auto_reply= 2,
    system_message='You are a comedian that tell bad jokes.'
)

[autogen.oai.client: 10-01 04:18:56] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [6]:
comedian.initiate_chat (
    audience,
    message="Welcome to my standup comedy show! Are you ready for a night full of laughter?",
    max_turns= 2
)

Comedian (to Audience):

Welcome to my standup comedy show! Are you ready for a night full of laughter?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Audience (to Comedian):

I guess we'll see if you can actually make us laugh. Show us what you've got.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Comedian (to Audience):

Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Audience (to Comedian):

*polite chuckle* Nice try, but that's an old one. Keep 'em coming.

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Welcome to my standup comedy show! Are you ready for a night full of laughter?', 'role': 'assistant', 'name': 'Comedian'}, {'content': "I guess we'll see if you can actually make us laugh. Show us what you've got.", 'role': 'user', 'name': 'Audience'}, {'content': 'Why did the scarecrow win an award? Because he was outstanding in his field!', 'role': 'assistant', 'name': 'Comedian'}, {'content': "*polite chuckle* Nice try, but that's an old one. Keep 'em coming.", 'role': 'user', 'name': 'Audience'}], summary="*polite chuckle* Nice try, but that's an old one. Keep 'em coming.", cost={'usage_including_cached_inference': {'total_cost': 0.0001945, 'gpt-3.5-turbo-0125': {'cost': 0.0001945, 'prompt_tokens': 215, 'completion_tokens': 58, 'total_tokens': 273}}, 'usage_excluding_cached_inference': {'total_cost': 0.0001945, 'gpt-3.5-turbo-0125': {'cost': 0.0001945, 'prompt_tokens': 215, 'completion_tokens': 58, 'total_tokens': 273}}}, human_in

## Code Executor

In [7]:
from google.colab import userdata

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

executor = LocalCommandLineCodeExecutor(
    timeout= 10,
    work_dir = 'code/'
)

code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER"
)

message_with_code_block = """
Here's a code that solves your problem:
```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create a DataFrame with sample data
data = pd.DataFrame({
    'x': np.arange(10),
    'y': np.random.randint(0, 10, 10)
})

# Plot the line plot
plt.plot(data['x'], data['y'])
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Line Plot')
plt.savefig('lineplot.png')
plt.show()

print('Line plot saved to lineplot.png')
```
"""

reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Figure(640x480)
Line plot saved to lineplot.png



In [8]:
code_assistant_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web,
2. When you need to perform some task with code,
use the code to perform the task and output the result. Finish the Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step When using code, you must indicate the script type in the code block.
The user cannot provide any other feedback If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code bo If the result indicates there is an error, fix the error and output the code again.
Suggest the full code instead When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible. Reply 'TERMINATE' in the end when everything is done.
"""

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get('OPENAI_API_KEY'),
    "temperature": 0.7
}

code_assistant = ConversableAgent(
    "assistant",
    llm_config=llm_config,
    system_message=code_assistant_system_message,
    code_execution_config= False #Turn off code execution for this agent
)


[autogen.oai.client: 10-01 04:19:12] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [9]:
chat_result = code_executor_agent.initiate_chat(
    code_assistant,
    message = "Write a Python code to calculate the 23rd prime number",
    max_turns = 2
)

code_executor_agent (to assistant):

Write a Python code to calculate the 23rd prime number

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to code_executor_agent):

To calculate the 23rd prime number, we can write a Python script that iterates through numbers starting from 2 and checks if each number is prime until we find the 23rd prime number. Here is the Python code:

```python
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

count = 0
num = 2

while True:
    if is_prime(num):
        count += 1
        if count == 23:
            print(num)
            break
    num += 1
```

This code defines a function `is_prime` to check if a number is prime and then iterates through numbers starting from 2 until it finds the 23rd prime number. When the 23rd prime number is found, it will be printed out. 


## Human in the loop

In [ ]:
from google.colab import userdata
from autogen import ConversableAgent

llm_config={
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get('OPENAI_API_KEY'),
    "temperature": 0.7
}

In [10]:
agent_guesser = ConversableAgent(
    name="agent_guesser",
    system_message="""Let's play a game. I have a person in mind, and you have to guess it.
    I'll respond with 'yes' or 'no' with a hint.
    You have 3 tries to guess the person's name. Don't ask for hints, just make best use of information you already
    Go ahead and start guessing!""",
    max_consecutive_auto_reply=1,
    llm_config=llm_config,
    human_input_mode="TERMINATE" # Ask for human input until the game is terminated.
)

agent_thinker = ConversableAgent(
    name="agent_thinker",
    system_message="""Let's play a game. Think of a famous personality, and I'll try to guess their name on every turn.
    Respond with 'yes' or 'no'. If no, also provide a hint. Let's get started!""",
    llm_config=llm_config,
    human_input_mode="NEVER" # Default.
)


[autogen.oai.client: 10-01 04:20:42] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-01 04:20:43] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [ ]:
result = agent_guesser.initiate_chat(
    agent_guesser,
    message="Let's start, I have someone in mind."
)

agent_guesser (to agent_guesser):

Let's start, I have someone in mind.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_guesser (to agent_guesser):

Is the person you are thinking of a fictional character?

--------------------------------------------------------------------------------
Please give feedback to agent_guesser. Press enter to skip and use auto-reply, or type 'exit' to stop the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
agent_guesser (to agent_guesser):

Yes, the person I have in mind is a fictional character.

--------------------------------------------------------------------------------
Please give feedback to agent_guesser. Press enter to skip and use auto-reply, or type 'exit' to stop the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
agent_guesser (to agent_guesser):

Is the fictional character you're thinking of known for their

## Tools

In [11]:
import requests
from bs4 import BeautifulSoup

def scrap_wiki_main_page(url: str) -> str:
  """
  Scrapes news content from wikipedia's main page.

  Args:
      url (str): The URL of the wikipedia page to scrape.

  Returns:
      str: The text content of the webpage.
         Returns None if there is an error during the process.
  """

  try:
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      content= soup.get_text()
      content = content[content.find("In the news"):content.find("Ongoing")]
      return content

    else:
      print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
      return None

  except Exception as e:
    print(f"An error occurred: {e}")
    return None

In [12]:
import os
from autogen import ConversableAgent

#Let's define the assistant agent that suggests tools to call
assistant = ConversableAgent(
    name="assistant",
    llm_config=llm_config,
    system_message="""You are a helpful AI news bot.
    You can pull content from wikipedia's main page.
    URL: https://en.wikipedia.org/wiki/Main_Page. Once pulled, only share the top 3 stories.
    Return 'TERMINATE' when the task is done."""
)

# The user proxy agent is used for interacting with the assistant agent and exeutes tool calls
user_proxy = ConversableAgent(
    name="user_proxy",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="NEVER",
)

#Register the tool's information with the assistant
assistant.register_for_llm(
    name="scrap_wiki_main_page",
    description="A tool for scraping today's news from wikipedia's main page",
) (scrap_wiki_main_page)

#Register the tool function with the proxy agent
user_proxy.register_for_execution(
    name="scrap_wiki_main_page"
)(scrap_wiki_main_page)



[autogen.oai.client: 10-01 04:20:57] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-01 04:20:57] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


<function __main__.scrap_wiki_main_page(url: str) -> str>

In [13]:
from autogen import Cache

with Cache.disk() as cache:
  chat_result = user_proxy.initiate_chat(
      assistant,
      message= "what's hot in todays news?",
      max_turns= 5
  )

user_proxy (to assistant):

what's hot in todays news?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to user_proxy):

***** Suggested tool call (call_S80VhGLXqQB8vdmPmPuFvTUg): scrap_wiki_main_page *****
Arguments: 
{"url":"https://en.wikipedia.org/wiki/Main_Page"}
*************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION scrap_wiki_main_page...
user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool (call_S80VhGLXqQB8vdmPmPuFvTUg) *****
In the news


Will Ashcroft

Flooding in Nepal leaves at least 193 people dead, including 37 in the nation's capital, Kathmandu.
In Australian rules football, the Brisbane Lions defeat the Sydney Swans to win the AFL Grand Final (Norm Smith Medal winner Will Ashcroft pictured).
Hezbollah leader Hassan Nasral

## Sequential Chat

In [14]:
# A quest where an adventurer seeks help from a wizard
# to get past the guardian of the secret vault

# Define the agents

# The Adventurer Agent seeks advice from a Wizard on magical matters.
adventurer_agent = ConversableAgent(
    name="adventurer_agent",
    system_message="""
    You are a worthy adventurous hero seeking advice from a Wizard on magical matters.
    Briefly communicate in less than 10 words.""",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

# The wizard agent is an expert in magic and provides magical advice to the Adventurer
wizard_agent= ConversableAgent(
    name="wizard_agent",
    system_message="""
    You are a wise and powerful Wizard. You provide magical advice to the Adventurer.
    You know the passcode to the vault is Enigma.
    Briefly communicate in less than 10 words.
    """,
    llm_config=llm_config,
    human_input_mode="NEVER"
)

#The guardian Agent guards the hidden gem and offer protection to the Adventurer.
guardian_agent = ConversableAgent(
    name="guardian_agent",
    system_message="""
    You are the Guardian of the Hidden Gem, sworn to protect it from unworthy adventurers.
    The passcode to the vault is Enigma. If adventurer tells you the passcode, say 'Welcome!' otherwise say
    'Incorrect Passcode'. Briefly communicate in less than 10 words.
    """,
    llm_config=llm_config,
    human_input_mode="NEVER"
)


[autogen.oai.client: 10-01 04:21:25] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-01 04:21:25] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-01 04:21:25] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [15]:
# Start a sequence of two agents chat
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat() method

sequence_chats =[
    {
        "recipient": wizard_agent,
        "message": "Help me get past the Guardian of the hidden gem in the secret vault.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm"
    },

    {
        "recipient": guardian_agent,
        "message": "Allow me to enter the secret vault!",
        "max_turns": 1,
        "summary_method": "reflection_with_llm"
    }
]

chat_result = adventurer_agent.initiate_chats(sequence_chats)


********************************************************************************
Starting a new chat....

********************************************************************************
adventurer_agent (to wizard_agent):

Help me get past the Guardian of the hidden gem in the secret vault.

--------------------------------------------------------------------------------
wizard_agent (to adventurer_agent):

Speak "Enigma" to pass the Guardian and claim the gem.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
adventurer_agent (to guardian_agent):

Allow me to enter the secret vault!
Context: 
Speak "Enigma" to pass the Guardian and claim the gem.

--------------------------------------------------------------------------------
guardian_agent (to adventurer_agent):

I

## Group Chat

In [16]:
# Define the roles and their descriptions
# a software engineering team:
# scrum master
# software engineer
# product manager

product_manager_agent= ConversableAgent(
    name="product_manager_agent",
    system_message= " You are an expert Product manager working in a scrum team of an IT company",
    llm_config=llm_config,
    human_input_mode="NEVER",
    description="Provides products requirements and priorities"
)

scrum_master_agent = ConversableAgent(
    name="scrum_master_agent",
    system_message="You are an expert Scrum master leadin th 14-day sprint in an IT company",
    llm_config=llm_config,
    human_input_mode="NEVER",
    description="Lead the sprint planning process"
)

software_engineer_agent = ConversableAgent(
    name="software_engineer_agent",
    system_message="You are an expert software engineer working in an IT company",
    llm_config=llm_config,
    human_input_mode="NEVER",
    description="Provides insight on technical feasability and effort estimation."
)

[autogen.oai.client: 10-01 04:21:53] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-01 04:21:54] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-01 04:21:54] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [17]:
allowed_transitions={
    product_manager_agent: [scrum_master_agent],
    scrum_master_agent: [product_manager_agent, software_engineer_agent],
    software_engineer_agent: [scrum_master_agent]
}


In [18]:
from autogen import GroupChat, GroupChatManager

# Create a group chat and provide the list of agents
sprint_planning_chat = GroupChat(
    agents=[product_manager_agent, scrum_master_agent, software_engineer_agent],
    messages=[],
    max_round=5,
    send_introductions=True,
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed"
)

#Create a GroupChatManager object that provides the GroupChat object as an input
spring_planning_chat_manager = GroupChatManager(
    groupchat=sprint_planning_chat,
    llm_config=llm_config,
)

# Initiate the chat
chat_result = product_manager_agent.initiate_chat(
    spring_planning_chat_manager,
    message= "We have a request to develop a website to track employee HR requests. What should we do?"
)

[autogen.oai.client: 10-01 04:22:00] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


product_manager_agent (to chat_manager):

We have a request to develop a website to track employee HR requests. What should we do?

--------------------------------------------------------------------------------

Next speaker: scrum_master_agent

scrum_master_agent (to chat_manager):

As the Scrum Master, I recommend that we start by breaking down the website development project into smaller tasks and user stories. This will help us prioritize the work and understand the requirements better. We can then estimate the effort required for each task with the help of the software engineers. Once we have a clear understanding of the scope and effort involved, we can create a sprint backlog and start working on the development in iterations. Let's prioritize the most important features for the initial release and plan our sprint accordingly. Do you have any specific requirements or features in mind for the website?

----------------------------------------------------------------------------


Next speaker: software_engineer_agent

software_engineer_agent (to chat_manager):

As a software engineer, I suggest starting by gathering detailed requirements for the employee HR request tracking website. This will help us define the scope of the project and understand the specific needs of the users. We should also consider factors such as user roles, permissions, data security, reporting capabilities, and integration with existing HR systems.

Once we have a clear understanding of the requirements, we can start working on the technical feasibility assessment. This involves evaluating the technologies, frameworks, and tools that will be best suited for building the website. We should also consider factors such as scalability, performance, and maintenance requirements during this evaluation.

Effort estimation is crucial for planning the development process effectively. We can break down the project into smaller tasks and estimate the effort required for each task. This will help us


Next speaker: software_engineer_agent

software_engineer_agent (to chat_manager):

It's great to see the teamwork and alignment among the team members. By following a structured and collaborative approach, the project to develop the employee HR request tracking website is off to a solid start. With clear requirements, technical feasibility assessment, and effort estimation in place, the team is well-equipped to plan and execute the development process effectively.

As the Scrum Master, your role in facilitating communication, removing impediments, and ensuring the team's focus on delivering value is crucial for the project's success. By working closely with the product manager and software engineers, you can guide the team through sprint planning, execution, and review, fostering a culture of collaboration and continuous improvement.

By leveraging the expertise of each team member and maintaining open communication channels, the team can overcome challenges, adapt to changes, and del

## Custom Speaker Selection

In [19]:
def custom_speaker_selection_function(last_speaker, groupchat):
  messages = groupchat.messages

  # Decide next turn
  if last_speaker is scrum_master_agent:
    return product_manager_agent
  elif last_speaker is software_engineer_agent:
    return scrum_master_agent
  elif last_speaker is product_manager_agent:
    return software_engineer_agent

  return None

sprint_planning_chat= GroupChat(
    agents=[product_manager_agent, scrum_master_agent, software_engineer_agent],
    messages=[],
    max_round=3,
    send_introductions=True,
    speaker_selection_method=custom_speaker_selection_function
)

spring_planning_chat_manager = GroupChatManager(
    groupchat=sprint_planning_chat,
    llm_config=llm_config,
)


# Initiate the chat with the product_manager as the starting speaker
chat_result = product_manager_agent.initiate_chat(
    spring_planning_chat_manager,
    message= "We have a request to develop a website to track employee HR requests. What should we do?",
    summary_method="reflection_with_llm"
)

[autogen.oai.client: 10-01 04:22:16] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


product_manager_agent (to chat_manager):

We have a request to develop a website to track employee HR requests. What should we do?

--------------------------------------------------------------------------------

Next speaker: software_engineer_agent

software_engineer_agent (to chat_manager):

As a software engineer, I would suggest starting by gathering detailed requirements for the HR request tracking website. This could include functionalities such as employee profile management, request submission, approval workflows, reporting capabilities, and user roles. 

Next, we can work on creating wireframes and mockups to visualize the user interface and user experience of the website. Once the design is finalized, we can move on to the development phase where we will implement the features according to the requirements.

During development, it's important to regularly communicate with the product manager and scrum master to ensure that the project is on track and meeting the desired obj

## Nested Chat


In [22]:
poetry_agent = ConversableAgent(
    name="poetry_agent",
    llm_config=llm_config,
    system_message="You are an AI poet. Create only one stanza.",
    human_input_mode="NEVER"
)

# Parameters of initiate_chat() method
sequence_chats=[
    {
        "recipient": wizard_agent,
        "max_turns": 1,
        "message": "Help me get past the Guardian of the hidden gem in the secret vault.",
        "summary_method": "reflection_with_llm",
        "summary_prompt": "Concisely summarize the instructions given by wizard agent to communicate with guardian."
    },
    {
        "recipient": guardian_agent,
        "max_turns": 1,
        "message": "Allow me to enter the secret vault!",
        "summary_method": "reflection_with_llm",
        "summary_prompt": "Concisely summarize the quest."
    },
    {
        "recipient": poetry_agent,
        "max_turns": 1,
        "message":"Write a poem on my expedition.",
        "summary_method": "last_msg",
    },
]

adventurer_agent.register_nested_chats(
    sequence_chats,
    # trigger determines if the agent should start nested chat given the sender agent.
    # In this case, the adventurer agent will not start the nested chats if the sender is
    # from the nested chats' recipient to avoid recursive calls.
    trigger=lambda sender: sender not in [wizard_agent, guardian_agent, poetry_agent]

)

[autogen.oai.client: 10-01 04:24:00] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [23]:
adventurer_agent.generate_reply(
    messages=[{"role": "user", "content": "Hello!"}]
)


********************************************************************************
Starting a new chat....

********************************************************************************
adventurer_agent (to wizard_agent):

Help me get past the Guardian of the hidden gem in the secret vault.

--------------------------------------------------------------------------------
wizard_agent (to adventurer_agent):

Speak "Enigma" to pass the Guardian and claim the gem.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
adventurer_agent (to guardian_agent):

Allow me to enter the secret vault!
Context: 
Speak "Enigma" to pass the Guardian and claim the gem.

--------------------------------------------------------------------------------
guardian_agent (to adventurer_agent):

I

'Through shadows deep and mysteries untold,\nThe adventurer agent seeks the gem of old,\nWhispers of Enigma to break the mold,\nIn the secret vault, a fortune to behold.'

## Image generation endpoints

In [24]:
!pip install pyautogen[llm] --upgrade -q

In [65]:
# snippets borrowed from https://microsoft.github.io/autogen/docs/notebooks/agentchat_image_generation_capability/
import os
import re

import traceback
import logging

from typing import Dict, Optional

from IPython.display import display
from PIL.Image import Image

import autogen
from autogen import (
    Agent,
    AssistantAgent,
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent
)
from autogen.agentchat.contrib import img_utils
from autogen.agentchat.contrib.capabilities import generate_images
from autogen.cache import Cache
from autogen.oai import openai_utils

from google.colab import userdata


In [74]:


# define configs for model
gpt_config = {
    "config_list": [{"model": "gpt-3.5-turbo", "api_key": userdata.get('OPENAI_API_KEY')}],
    "timeout": 120,
    "temperature": 0.5,
}

gpt_vision_config = {
    "config_list": [{"model": "gpt-4o", "api_key": userdata.get('OPENAI_API_KEY')}],
    "timeout": 120,
    "temperature": 0.5,
}

dalle_config = {
    "config_list": [{"model": "dall-e-2", "api_key": userdata.get('OPENAI_API_KEY')}],
    "timeout": 120,
    "temperature": 0.5,
}




In [75]:
# create image generator agent
agent = autogen.ConversableAgent(
    name="agent_with_dalle_capability",
    llm_config=gpt_vision_config,
    max_consecutive_auto_reply=2,
    human_input_mode="NEVER"
)

# define image generation capability
dalle_gen = generate_images.DalleImageGenerator(llm_config=dalle_config, resolution='256x256')
image_gen_capability = generate_images.ImageGeneration(
    image_generator=dalle_gen,
    text_analyzer_llm_config=gpt_config,
    verbosity=2
)

# add ability
image_gen_capability.add_to_agent(agent)

[autogen.oai.client: 10-01 07:12:50] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-01 07:12:50] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [76]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config = False,
    human_input_mode="NEVER",
)

In [77]:
reply = user_proxy.initiate_chat(agent, message = "imagine kittens playing with a ball of yarn", max_turns=1)
reply

user_proxy (to agent_with_dalle_capability):

imagine kittens playing with a ball of yarn

--------------------------------------------------------------------------------
agent_with_dalle_capability (to user_proxy):

That sounds adorable! Here is an image of kittens playing with a ball of yarn:

![Kittens playing with a ball of yarn](https://i.imgur.com/n2h6M3V.jpg)

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'imagine kittens playing with a ball of yarn', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'That sounds adorable! Here is an image of kittens playing with a ball of yarn:\n\n![Kittens playing with a ball of yarn](https://i.imgur.com/n2h6M3V.jpg)', 'role': 'user', 'name': 'agent_with_dalle_capability'}], summary='That sounds adorable! Here is an image of kittens playing with a ball of yarn:\n\n![Kittens playing with a ball of yarn](https://i.imgur.com/n2h6M3V.jpg)', cost={'usage_including_cached_inference': {'total_cost': 0.0008100000000000001, 'gpt-4o-2024-05-13': {'cost': 0.0008100000000000001, 'prompt_tokens': 39, 'completion_tokens': 41, 'total_tokens': 80}}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])

In [54]:
def extract_images(sender, recipient):
    images = []
    all_messages = sender.chat_messages[recipient]

    for message in reversed(all_messages):
        # The GPT-4V format, where the content is an array of data
        contents = message.get("content", [])
        for content in contents:
            if isinstance(content, str):
                continue
            if content.get("type", "") == "image_url":
                img_data = content["image_url"]["url"]
                images.append(img_utils.get_pil_image(img_data))

    if not images:
        raise ValueError("No image data found in messages.")

    return images


In [58]:
images = extract_images(user_proxy, agent)
display(images[0])

ValueError: No image data found in messages.